<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/code_clones_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .
%cd ..

Cloning into 'fastText'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3197 (delta 0), reused 1 (delta 0), pack-reused 3195
Receiving objects: 100% (3197/3197), 7.84 MiB | 33.46 MiB/s, done.
Resolving deltas: 100% (2007/2007), done.
/content/fastText
Processing /content/fastText
  Stored in directory: /tmp/pip-ephem-wheel-cache-ub3_uu_5/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
/content


In [0]:
import fastText

In [0]:
%cd /content/gdrive/My\ Drive/code-clones

/content/gdrive/My Drive/code-clones


In [0]:
import xmlextractor
import pandas as pd
import numpy as np

In [0]:
duplicates = xmlextractor.parseXml("data/lwjgl-master/duplicates.xml", "data/lwjgl-master")
duplicates.extend(xmlextractor.parseXml("data/lwjgl3-master/duplicates.xml", "data/lwjgl3-master"))
duplicates.extend(xmlextractor.parseXml("data/spring-framework-master/duplicates_marked.xml", "data/spring-framework-master"))
duplicates.extend(xmlextractor.parseXml("data/jenkins-master/duplicates_marked.xml", "data/jenkins-master"))
duplicates.extend(xmlextractor.parseXml("data/RxJava-2.x/duplicates_marked.xml", "data/RxJava-2.x"))

In [0]:
import re

all_fragment_duplicates = []
for duplicate in duplicates:
    text = ''.join(duplicate.fragments)
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
    text = re.sub(" +", " ", text)
    all_fragment_duplicates.append({'text': text, 'exp': duplicate.exp})

In [0]:
all_fragment_duplicates[0]['text']

'/** * A hash map using primitive ints as keys rather than objects. * * @author Justin Couch * @author Alex Chaffee (alex@apache.org) * @author Stephen Colebourne * @author Nathan Sweet */ final class FastIntMap<V> implements Iterable<FastIntMap.Entry<V>> { private Entry[] table; private int size, mask, capacity, threshold; /** Same as: FastIntMap(16, 0.75f); */ FastIntMap() { this(16, 0.75f); } /** Same as: FastIntMap(initialCapacity, 0.75f); */ FastIntMap(int initialCapacity) { this(initialCapacity, 0.75f); } FastIntMap(int initialCapacity, float loadFactor) { if ( initialCapacity > 1 << 30 ) throw new IllegalArgumentException("initialCapacity is too large."); if ( initialCapacity < 0 ) throw new IllegalArgumentException("initialCapacity must be greater than zero."); if ( loadFactor <= 0 ) throw new IllegalArgumentException("initialCapacity must be greater than zero."); capacity = 1; while ( capacity < initialCapacity ) capacity <<= 1; this.threshold = (int)(capacity * loadFactor); t

In [0]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

def resample(X, y):
  rus = RandomUnderSampler(random_state=23)
  return rus.fit_resample(X, y)

In [0]:
df = pd.DataFrame(all_fragment_duplicates)
X = np.array(df['text'])
y = np.array(df['exp'])
X_resampled, y_resampled = resample(X.reshape(-1, 1), y.reshape(-1, 1))
X_resampled = X_resampled.reshape(1, -1)[0]
y_resampled = y_resampled.reshape(1, -1)[0]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

In [0]:
train_text = '\n\r'.join(['__label__{0} {1}'.format(y_train[i], X_train[i]) for i in range(len(X_train))])

In [0]:
input_file_name = 'train.txt'
with open(input_file_name, 'w') as f:
  f.write(train_text)

In [0]:
!head train.txt

__label__0  runs.put(3, previousPreviousBuild); TestBuild previousBuild = new TestBuild(project, Result.SUCCESS, 15, previousPreviousBuild); runs.put(2, previousBuild); TestBuild lastBuild = new TestBuild(project, Result.SUCCESS, 42, previousBuild); runs.put(1, lastBuild); runs.put(6, prev5Build); TestBuild prev4Build = new TestBuild(project, Result.SUCCESS, 1, prev5Build); runs.put(5, prev4Build); TestBuild prev3Build = new TestBuild(project, Result.SUCCESS, 1, prev4Build); runs.put(4, prev3Build);
__label__0  protected BufferObject(final int type, final int usage, final Buffer buffer) { this(type, usage); setData(buffer); } protected BufferObject(final int type, final int usage, final int dataSize) { this(type, usage); setData(dataSize); }
__label__1  assertEquals("Invalid content-length", outputMessage.getBodyAsBytes().length, outputMessage.getHeaders().getContentLength()); assertEquals("Invalid content-length", outputMessage.getBodyAsBytes().length, outputMessage.getHeaders().getCo

In [0]:
!ls

csv  data  log.txt  __pycache__  test.txt  train.txt  xmlextractor.py


In [0]:
#./fasttext supervised -input train.txt -output model
fastText_model = fastText.train_supervised(input_file_name)

In [0]:
test_file_name = 'test.txt'
y_pred = []
for test_text in X_test:
  with open(test_file_name, 'w') as f:
    f.write(test_text)
  #./fasttext predict model.bin test.txt
  pred = fastText_model.predict(test_file_name)
  y_pred.append(pred)

In [0]:
!head test.txt

method.getName().equals("equals") && args != nullmethod.getName().equals("setClientID") && args != nullmethod.getName().equals("setExceptionListener") && args != nullmethod.getName().equals("equals") && args != null

In [0]:
y_pred_exp = np.array([int(y[0][0].replace('__label__', '')) for y in y_pred])

In [0]:
y_pred_exp

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
def print_uniq_count(arr):
  unique, counts = np.unique(arr, return_counts=True)
  print(np.asarray((unique, counts)).T)

In [0]:
print_uniq_count(y_pred_exp)

[[   1 2715]]


In [0]:
def calc_fnr_fpr(y_target, y_prediction):
  fp = 0
  fn = 0
  n = len(y_target)
  for i in range(n):
    if y_prediction[i] == 1 and y_target[i] == 0:
      fp += 1
    if y_prediction[i] == 0 and y_target[i] == 1:
      fn += 1
  return (fn / n, fp / n) 

In [0]:
calc_fnr_fpr(y_test, y_pred_exp)

(0.0, 0.49060773480662984)